In [100]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import re
import json

In [101]:
BASE_URL = 'https://en.wikipedia.org'
HEADERS = {'User-Agent': 'Mozilla/5.0'}

# Fetch and parse the HTML content of a given Wikipedia page
def get_soup(url):
    response = requests.get(url, headers=HEADERS)
    return BeautifulSoup(response.content, "lxml")

In [102]:
soup = get_soup(BASE_URL + '/wiki/List_of_highest-grossing_films')
tables = soup.find_all('table', {'class': 'wikitable'})

# Finding the main table
target_table = None
for table in tables:
    caption = table.find("caption")
    if caption and "high-grossing films by year" in caption.text.lower():
        target_table = table
        break  # Stop once we find the correct table

In [103]:
def extract_movie_info(movie_url):
    # Extracts information from the movie's Wikipedia infobox and cleaning them
    soup = get_soup(movie_url)
    infobox = soup.find("table", {"class": "infobox vevent"})

    if not infobox:
        return {}

    details = {}
    rows = infobox.find_all("tr")

    for row in rows:
        header = row.find("th")
        value = row.find("td")

        if header and value:
            # Merge multi-line keys
            for br in header.find_all("br"):
                br.replace_with(" ")

            key = header.text.strip()

            # Delete references from text
            for sup in value.find_all("sup"):
                sup.decompose()
            
            # Replace <br>(\n) tags with commas
            for br in value.find_all("br"):
                br.replace_with(", ")

            # Check if the value contains a list
            if value.find("ul") or value.find("ol"):
                val = ", ".join([li.text.strip() for li in value.find_all("li")])  # Join list items with a comma
            else:
                # Replace <br> and remove \n (newlines) from general text
                for br in value.find_all("br"):
                    br.replace_with(" ")

                val = value.text.strip().replace("\n", " ")


            # Store only relevant information
            if key in ["Directed by", "Production company", "Production companies", "Distributed by", "Running time", "Country", "Countries", "Languages", "Language"]:
                details[key] = val
    return details

In [104]:
def clean_money_value(value):
    """Extracts the highest numeric value from budget or box office data."""
    if not value:
        return None  # Handle missing values

    value = value.replace(",", "").replace(" ", "")  # Remove commas and spaces

    match = re.search(r"\$([\d]+(?:\.\d+)?)(?:–\$([\d]+(?:\.\d+)?))?", value)  # Match number ranges
    if match:
        return float(match.group(2)) if match.group(2) else float(match.group(1))  # Take the higher value in range

    return None  # Return None if no valid number is found

In [105]:
def get_movie_rows(table):
    """Extracts movies from the Wikipedia table and fetches additional info from their Wikipedia pages."""
    rows = table.find_all("tr")[1:]  # Skip header
    movies = []

    for row in rows:
        columns = row.find_all(["th", "td"])  # Since year uses <th> and the rest uses <td>
        if len(columns) < 5:  # Ensure enough columns exist
            continue  

        # Extract year
        year_text = columns[0].text.strip()
        match = re.search(r"\b\d{4}\b", year_text)  # Match a 4-digit number (like 1997)
        year = int(match.group()) if match else None  # Convert only if valid
        if year is None:
            continue  # Skip rows without a valid year

        # Extract title and link
        title = columns[1].text.strip()
        link_tag = columns[1].find("a")
        movie_link = BASE_URL + link_tag["href"] if link_tag else None

        # Extract budget and box office
        box_office = clean_money_value(columns[2].text.strip()) if len(columns) > 2 else None
        budget = clean_money_value(columns[3].text.strip()) if len(columns) > 3 else None

        # Extract additional details from the movie's page
        movie_details = extract_movie_info(movie_link) if movie_link else {}

        movies.append({
            "year": year,
            "title": title,
            "Budget": budget,
            "Box office": box_office,
            **movie_details
        })

    return movies

In [106]:
movies_data = get_movie_rows(target_table)
# print(movies_data)

In [107]:
# Define database file
DB_FILE = "movies.db"

# Connect to SQLite database
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

# Create table
cursor.execute("""
CREATE TABLE IF NOT EXISTS films (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    release_year INTEGER,
    director TEXT,
    box_office REAL,
    country TEXT,
    production_company TEXT,
    distributed_by TEXT,
    running_time TEXT,
    languages TEXT,
    budget REAL
);
""")
conn.commit()


In [108]:
def insert_movies(movies_data):
    for movie in movies_data:
        cursor.execute("""
        INSERT INTO films (title, release_year, director, box_office, country, 
                           production_company, distributed_by, running_time, 
                           languages, budget)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            movie["title"],
            movie.get("year"),
            movie.get("Directed by"),
            movie.get("Box office"),
            movie.get("Country") or movie.get("Countries"),
            movie.get("Production company") or movie.get("Production companies"),
            movie.get("Distributed by"),
            movie.get("Running time"),
            movie.get("Languages") or movie.get("Language"),
            movie.get("Budget")
        ))
    conn.commit()

In [109]:
insert_movies(movies_data)
conn.close()

In [110]:
# Connect to the sql database
OUTPUT_JSON = "movies.json"
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

# Get column names dynamically
cursor.execute("PRAGMA table_info(films)")
columns = [col[1] for col in cursor.fetchall()]  # Extract column names

# Fetch all data
cursor.execute("SELECT * FROM films")
movies = [dict(zip(columns, row)) for row in cursor.fetchall()]

# Save to JSON
with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(movies, f, indent=4, ensure_ascii=False)

conn.close()